### imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import clip
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import pickle as pkl
import numpy as np 
import xgboost as xgb
from sklearn.model_selection import train_test_split


print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1
CUDA available: False
CUDA version: None


### utils

In [2]:
# def create_conv1d(input_dim=512, kernel_size=3, padding=1, stride=1, num_layers=3, channels_per_layer=[256, 128, 64]):
#     print(f"Input size for Conv1D: {input_dim}")
#     if len(channels_per_layer) != num_layers:
#         raise ValueError("The number of channels per layer must match the number of layers")
#     layers = []

#     layers.append(nn.Conv1d(in_channels=input_dim, out_channels=channels_per_layer[0], kernel_size=kernel_size, padding=padding, stride=stride))
#     layers.append(nn.BatchNorm1d(channels_per_layer[0]))
#     layers.append(nn.ReLU())

#     in_channels = channels_per_layer[0]
#     for out_channels in channels_per_layer[1:]:
#         layers.append(nn.Conv1d(
#             in_channels=in_channels, 
#             out_channels=out_channels, 
#             kernel_size=kernel_size, 
#             padding=padding, 
#             stride=stride
#         ))
#         layers.append(nn.BatchNorm1d(out_channels)) #NORMALIZACIÓN
#         layers.append(nn.ReLU())
        
#         in_channels = out_channels

#     model = nn.Sequential(*layers)

#     model = nn.Sequential(*layers)
#     print(f"Output size for Conv1D: {channels_per_layer[-1]}")  

#     return model


# def create_mlp(input_dim=532, num_layers=3, hidden_per_layer=[256, 128, 64], output_dim=1):
#     print(f"Input size for MLP: {input_dim}")
#     if len(hidden_per_layer) != num_layers:
#         raise ValueError("The number of hidden units per layer must match the number of layers")
    
#     layers = []
#     in_features = input_dim
#     for out_features in hidden_per_layer:
#         layers.append(nn.Linear(in_features, out_features))
#         layers.append(nn.ReLU())  # Usamos ReLU como función de activación en las capas ocultas
#         in_features = out_features
    
#     layers.append(nn.Linear(in_features, output_dim))    
#     model = nn.Sequential(*layers)
#     return model

### models

In [3]:


class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, n_layers, output_dim=1):
        super(NeuralNetwork, self).__init__()
        layers = []
        current_dim = input_dim

        for layer in n_layers:
            layers.append(nn.Linear(current_dim, layer))
            layers.append(nn.ReLU())
            current_dim = layer

        layers.append(nn.Linear(current_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class MLP:
    def __init__(self, input_dim, n_layers=[70, 40, 25, 10], learning_rate=0.01, epochs=1000, batch_size=500, results=True):
        self.input_dim = input_dim
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = NeuralNetwork(input_dim, n_layers)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.patience = 10
        self.results = results
        self.criterion = nn.MSELoss()  # Define the loss function

    def rmse(self, outputs, targets):
        return torch.sqrt(self.criterion(outputs, targets))

    def fit(self, X_train, y_train, X_val=None, y_val=None):
        self.model.train()
        X_train_tensor = X_train  #torch.tensor(X_train.values, dtype=torch.float32)
        y_train_tensor = y_train #torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

        if X_val is not None and y_val is not None:
            X_val_tensor = X_val#torch.tensor(X_val.values, dtype=torch.float32)
            y_val_tensor = y_val #torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

        best_val_loss = float('inf')  # Initialize the best validation loss
        early = 0

        # Batch training loop
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()

            # Iterate over batches
            for i in range(0, len(X_train_tensor), self.batch_size):
                # Create mini-batch slices
                X_batch = X_train_tensor[i:i + self.batch_size]
                y_batch = y_train_tensor[i:i + self.batch_size]

                # Forward pass
                outputs = self.model(X_batch)
                loss = self.rmse(outputs, y_batch)

                # Backward pass
                loss.backward()
                self.optimizer.step()

            if X_val is not None and y_val is not None:
                with torch.no_grad():
                    val_outputs = self.model(X_val_tensor)
                    val_loss = self.rmse(val_outputs, y_val_tensor)

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early = 0
                else:
                    early += 1

            # if early >= self.patience:
            #     print(f'Early stopping at epoch {epoch+1}')
            #     break

            if epoch % 10 == 0 and self.results:
                if X_val is not None and y_val is not None:
                    print(f'Epoch {epoch+1}/{self.epochs}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')
                else:
                    print(f'Epoch {epoch+1}/{self.epochs}, Train Loss: {loss.item():.4f}')

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X.values, dtype=torch.float32)
        with torch.no_grad():
            predictions = self.model(X_tensor).numpy()
        return predictions

    def score(self, y_test, y_pred, metric="RMSE"):
        if metric == 'MSE':
            return mean_squared_error(y_test, y_pred)
        elif metric == 'MAE':
            return mean_absolute_error(y_test, y_pred)
        elif metric == 'R2':
            return r2_score(y_test, y_pred)
        elif metric == 'RMSE':
            return np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            raise ValueError("Metric not valid")


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def create_conv1d(input_dim, kernel_size, padding, stride, num_layers, channels_per_layer, dropout_prob=0.3):
    """
    Crea una red convolucional 1D con capas de normalización y Dropout.
    """
    layers = []
    for i in range(num_layers):
        in_channels = input_dim if i == 0 else channels_per_layer[i - 1]
        out_channels = channels_per_layer[i]

        layers.append(nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding))
        layers.append(nn.BatchNorm1d(out_channels))  # Normalización
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_prob))  # Dropout

    return nn.Sequential(*layers)

def create_mlp(input_dim, num_layers, hidden_per_layer, dropout_prob=0.3):
    """
    Crea una red completamente conectada (MLP) con capas de normalización y Dropout.
    """
    layers = []
    for i in range(num_layers):
        in_features = input_dim if i == 0 else hidden_per_layer[i - 1]
        out_features = hidden_per_layer[i]

        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(out_features))  # Normalización
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_prob))  # Dropout

    return nn.Sequential(*layers)

class CombinedModel(nn.Module):
    def __init__(self, input_dim_images=512, input_dim_tabular=17, kernel_size_images=3,
                 kernel_size_tabular=3, padding=1, stride=1, num_layers_1=2, num_layers_2=2, 
                 channels_per_layer_1=[12, 20], channels_per_layer_2=[12, 20], mlp_hidden=[500, 250],
                 dropout_prob=0.3):
        """
        Modelo combinado que usa una red convolucional para datos de imagen y un MLP para datos tabulares.
        """
        super(CombinedModel, self).__init__()

        mlp_input_dim = channels_per_layer_1[-1] + channels_per_layer_2[-1]

        self.conv1 = create_conv1d(
            input_dim=input_dim_images, 
            kernel_size=kernel_size_images, 
            padding=padding, 
            stride=stride, 
            num_layers=num_layers_1, 
            channels_per_layer=channels_per_layer_1,
            dropout_prob=dropout_prob
        )

        self.conv2 = create_conv1d(
            input_dim=input_dim_tabular, 
            kernel_size=kernel_size_tabular, 
            padding=padding, 
            stride=stride, 
            num_layers=num_layers_2, 
            channels_per_layer=channels_per_layer_2,
            dropout_prob=dropout_prob
        )

        self.mlp = create_mlp(
            input_dim=mlp_input_dim, 
            num_layers=len(mlp_hidden), 
            hidden_per_layer=mlp_hidden, 
            dropout_prob=dropout_prob
        )

    def forward(self, X_images, X_tabular):
        """
        Propagación hacia adelante.
        """
        x1 = self.conv1(X_images)
        x2 = self.conv2(X_tabular)

        # Aplanar
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)

        # Combinar y pasar por el MLP
        x_combined = torch.cat((x1, x2), dim=1)
        output = self.mlp(x_combined)

        return output

    def fit(self, X_images_train, X_tabular_train, Y_train, epochs=10, batch_size=32, learning_rate=0.001, loss_fn=F.mse_loss):
        """
        Entrena el modelo con los datos proporcionados.
        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        train_dataset = torch.utils.data.TensorDataset(X_images_train, X_tabular_train, Y_train)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Entrenamiento
        for epoch in range(epochs):
            self.train()
            running_loss = 0.0
            
            for batch_idx, (X_images, X_tabular, y) in enumerate(train_loader):
                optimizer.zero_grad()
                outputs = self(X_images, X_tabular)
                loss = loss_fn(outputs, y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            avg_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")

# class CombinedModel(nn.Module):
#     def __init__(self, input_dim_images=512, input_dim_tabular=17, kernel_size_images=3,
#                  kernel_size_tabular=3, padding=1, stride=1, num_layers_1=2, num_layers_2=2, 
#                  channels_per_layer_1=[12,20], channels_per_layer_2=[12, 20], mlp_hidden=[500, 250]):

#         super(CombinedModel, self).__init__()
#         mlp_input_dim = channels_per_layer_1[-1] + channels_per_layer_2[-1]
#         self.conv1 = create_conv1d(input_dim=input_dim_images, kernel_size=kernel_size_images, padding=padding, stride=stride, num_layers=num_layers_1, channels_per_layer=channels_per_layer_1)
#         self.conv2 = create_conv1d(input_dim=input_dim_tabular, kernel_size=kernel_size_tabular, padding=padding, stride=stride, num_layers=num_layers_2, channels_per_layer=channels_per_layer_2)
#         self.mlp = create_mlp(input_dim=mlp_input_dim, num_layers=len(mlp_hidden), hidden_per_layer=mlp_hidden)

#     def forward(self, X_images, X_tabular):

#         x1 = self.conv1(X_images)
#         x2 = self.conv2(X_tabular)
#         x1 = x1.view(x1.size(0), -1)  # Aplanar
#         x2 = x2.view(x2.size(0), -1)  # Aplanar

#         x_combined = torch.cat((x1, x2), dim=1)  # Concatenación
#         output = self.mlp(x_combined)

#         return output

#     def fit(self, X_images_train, X_tabular_train, Y_train, epochs=10, batch_size=32, learning_rate=0.001, loss_fn=F.mse_loss):
#         optimizer = optim.Adam(self.parameters(), lr=learning_rate)

#         train_dataset = torch.utils.data.TensorDataset(X_images_train, X_tabular_train, Y_train)
#         train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#         # Entrenamiento
#         for epoch in range(epochs):
#             self.train()
#             running_loss = 0.0
            
#             for batch_idx, (X_images, X_tabular, y) in enumerate(train_loader):
#                 optimizer.zero_grad()
#                 outputs = self(X_images, X_tabular)
#                 loss = loss_fn(outputs, y)
#                 loss.backward()
#                 optimizer.step()
#                 running_loss += loss.item()
            
#             avg_loss = running_loss / len(train_loader)
#             print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")


### perdictions

In [13]:
# model = CombinedModel(input_dim_images=512, input_dim_tabular=X_tabular_train.shape[1])
# model.fit(X_images_train, X_tabular_train, y_train, epochs=10, batch_size=100, learning_rate=0.000001, loss_fn=F.mse_loss)

X_train = pd.read_csv('../X_train.csv')
X_val = pd.read_csv('../X_val.csv')
X_test = pd.read_csv('../X_test.csv')
y_train = pd.read_csv('../y_train.csv')
y_test = pd.read_csv('../y_test.csv')
y_val = pd.read_csv('../y_val.csv')

#transform to tensor
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_val = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

#normalize with train max
X_train = X_train / X_train.max()
X_val = X_val / X_train.max()
X_test = X_test / X_train.max()

#separar en validation

input_dim = X_train.shape[1]  # El número de características (512)
n_layers = [255, 128, 32]  # Definir las capas ocultas
mlp_model = MLP(input_dim=input_dim, n_layers=n_layers, epochs=200000, learning_rate=0.0005, batch_size=1500)

# # Ajustar el modelo
mlp_model.fit(X_train, y_train, X_val, y_val)

plt.plot(y_test_mlp, y_pred, 'o')

# Hacer predicciones con el modelo entrenado


Epoch 1/200000, Train Loss: nan, Validation Loss: nan
Epoch 11/200000, Train Loss: nan, Validation Loss: nan
Epoch 21/200000, Train Loss: nan, Validation Loss: nan
Epoch 31/200000, Train Loss: nan, Validation Loss: nan
Epoch 41/200000, Train Loss: nan, Validation Loss: nan
Epoch 51/200000, Train Loss: nan, Validation Loss: nan
Epoch 61/200000, Train Loss: nan, Validation Loss: nan
Epoch 71/200000, Train Loss: nan, Validation Loss: nan
Epoch 81/200000, Train Loss: nan, Validation Loss: nan
Epoch 91/200000, Train Loss: nan, Validation Loss: nan
Epoch 101/200000, Train Loss: nan, Validation Loss: nan
Epoch 111/200000, Train Loss: nan, Validation Loss: nan
Epoch 121/200000, Train Loss: nan, Validation Loss: nan
Epoch 131/200000, Train Loss: nan, Validation Loss: nan
Epoch 141/200000, Train Loss: nan, Validation Loss: nan
Epoch 151/200000, Train Loss: nan, Validation Loss: nan
Epoch 161/200000, Train Loss: nan, Validation Loss: nan
Epoch 171/200000, Train Loss: nan, Validation Loss: nan
Epo

KeyboardInterrupt: 